In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
#path  = 'C:/PycharmProjects/pythonProject1/yjz/Extension for hts/JayCode/Data/'
tr_y = pd.read_pickle("past_1913.pkl")
ts_y = pd.read_pickle("future_28.pkl")
y_hist   = tr_y['y']
y_test   = ts_y['y']
fcst = pd.read_pickle("fcsts_base2cohe_tdfp.pkl")

In [5]:
ts_y['unique_id'].iloc[-28:].unique()

array(['TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516'],
      dtype=object)

In [2]:
fcst

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,37936.346099,37929.870202,37936.346099,37942.131159,42166.539062,42617.360970,42166.539062,42907.588644
1,35346.413219,36220.831410,35346.413219,35188.999335,42043.816406,41899.184619,42043.816406,37509.946040
2,33967.281969,34798.572205,33967.281969,34482.372855,41803.699219,40998.170227,41803.699219,38292.632633
3,34286.477860,34825.058046,34286.477860,34865.203322,41535.496094,40136.064270,41535.496094,38456.118033
4,38999.291696,39600.392103,38999.291696,39944.773472,41288.902344,39528.916325,41288.902344,40531.125660
...,...,...,...,...,...,...,...,...
1199515,0.177103,0.177103,0.062438,0.096519,0.076043,0.076043,0.040844,0.028109
1199516,0.186536,0.186536,0.078134,0.113224,0.074525,0.074525,0.035981,0.068262
1199517,0.174125,0.174125,0.073140,0.089249,0.074324,0.074324,0.037610,0.098300
1199518,0.242970,0.242970,0.098994,0.145623,0.073504,0.073504,0.039887,0.127565


## Functions for RMSSE

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse

def _get_df(df:pd.DataFrame, n:int = 42840, gap:int=28):
    df_csv = []
    for i in tqdm(range(n)):
        df_csv.append(df[i*gap:(i+1)*gap].reset_index(drop=True))
    df_csv = pd.concat(df_csv,axis=1,ignore_index= True)
    return df_csv
def rmsse(yts, fcst, ytr, n:int = 42840):
    df_l = []
    scaler =  (ytr.diff().dropna()**2).mean(axis = 0)
    for i in tqdm(range(n)):
        mse_ = mse(yts.iloc[:,i],fcst.iloc[:,i])
        df_l.append(np.sqrt(mse_/scaler[i]))
    return df_l   
y_tr = _get_df(y_hist,gap = 1913)
y_ts = _get_df(y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:03<00:00, 13686.78it/s]


## RMSSE:

In [4]:
i = 'lgb_base'
df = _get_df(fcst[i])
pd.DataFrame({f'{i}_rmsse':rmsse(yts = y_ts,fcst=df,ytr = y_tr)})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2592.10it/s]


,lgb_base_rmsse
0,0.381607
1,0.386990
2,0.482995
3,0.542488
4,0.530162
...,...
42835,1.079308
42836,0.540944
42837,0.621724
42838,2.326800


In [5]:
%%time
fcst = pd.read_pickle("fcsts_base2cohe_tdfp.pkl")
rmsse_l = []
for i in fcst.columns:
    df = _get_df(fcst[i])
    df = pd.DataFrame({f'{i}_rmsse':rmsse(yts = y_ts,fcst=df,ytr = y_tr)})
    rmsse_l.append(df)
    print(f"{i} Finished！")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2596.51it/s]


lgb_base Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2596.74it/s]


lgb_bu Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2606.29it/s]


...

ets_bu Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2603.80it/s]


ets_td Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:16<00:00, 2607.72it/s]

ets_mint Finished！
Wall time: 2min 56s


In [6]:
all_r = pd.concat(rmsse_l,axis=1)
all_r.to_pickle("h1_RMSSEs_tdfp.pkl")
all_r

,lgb_base_rmsse,lgb_bu_rmsse,lgb_td_rmsse,lgb_mint_rmsse,ets_base_rmsse,ets_bu_rmsse,ets_td_rmsse,ets_mint_rmsse
0,0.381607,0.338737,0.381607,0.265738,1.147773,1.139346,1.147773,0.765801
1,0.386990,0.365493,0.437034,0.278792,1.171681,1.181807,1.213860,0.755382
2,0.482995,0.499492,0.547123,0.439630,0.921631,0.904829,0.976054,0.664014
3,0.542488,0.602730,0.521796,0.444979,1.233625,1.192344,1.192668,0.930212
4,0.530162,0.439505,0.366426,0.250524,1.143070,1.114839,1.149127,0.742798
...,...,...,...,...,...,...,...,...
42835,1.079308,1.079308,1.055871,1.069534,0.989247,0.989247,1.040497,0.995505
42836,0.540944,0.540944,0.505536,0.528959,0.500271,0.500271,0.501746,0.508833
42837,0.621724,0.621724,0.534016,0.573881,0.538230,0.538230,0.526341,0.543464
42838,2.326800,2.326800,2.249731,2.297882,2.228416,2.228416,2.257921,2.236254


In [7]:
all_r["lgb_td_rmsse"].max()

13.798282142206059

In [12]:
pd.read_pickle("h1_h28_RMSSE.pkl")["lgb_td_rmsse"].max()

19.036921056900677

In [8]:
(1.213860+0.976054+1.192668)/3

1.1275273333333333